## Step-1 First verify the given keras model is working fine 

In [8]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from local_utils import detect_lp
from os.path import splitext,basename
from tensorflow.keras.models  import model_from_json
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
import glob
import os
import json
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def save_model_h5_to_tf_format(path):
    try:
        path = splitext(path)[0]
        #print(splitext(path.split('/')[0]))
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        #print(os.path())
        # Save the model to h5 format
        model.save("models/wpod_net_all_in_one.h5")
        #print("Keras Model Saved successfully as h5 format")
        model = tf.keras.models.load_model('models/wpod_net_all_in_one.h5')
        # Save the model to TF SavedModel format
        tf.saved_model.save(model, "models")
        #print("successfully saved keras model h5 file to tensorflow SavedModel format")
        return 
    except Exception as e:
        print(e)

def get_plate(image_path, Dmax=608, Dmin = 608):
    vehicle = preprocess_image(image_path)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    _ , LpImg, _, cor = detect_lp(tf_model, vehicle, bound_dim, lp_threshold=0.5)
    return vehicle, LpImg, cor

def preprocess_image(image_path,resize=False):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

# Create sort_contours() function to grab the contour of each digit from left to right
def sort_contours(cnts,reverse = False):
    i = 0
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
                                        key=lambda b: b[1][i], reverse=reverse))
    return cnts

# pre-processing input images and pedict with model
def predict_characters_from_model(image):
    # Load model architecture, weight and labels
    json_file = open('models/character_recoginition/MobileNets_character_recognition.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights("models/character_recoginition/License_character_recognition_weight.h5")
    #print("[INFO] Model loaded successfully...")
    labels = LabelEncoder()
    labels.classes_ = np.load('models/character_recoginition/license_character_classes.npy')
    #print("[INFO] Labels loaded successfully...")

    image = cv2.resize(image,(80,80))
    image = np.stack((image,)*3, axis=-1)
    prediction = labels.inverse_transform([np.argmax(model.predict(image[np.newaxis,:]))])
    return prediction



In [10]:


# # wpod_net_path = "models/wpod-net.json"
# save_model_h5_to_tf_format(wpod_net_path)

# # Just provide the directory name where the TF *.pb (model file saved_model.pb) file is located
# tf_model = keras.models.load_model('models')
# #print(tf_model.outputs)
# #print(tf_model.inputs)

# #print("Loading vehicle image for License Plate Detection...")
# input_image_path = "dataset/plate3.jpeg"

# #print("Detecting License Plate...")
# vehicle, LpImg, cor = get_plate(input_image_path)

# #fig = plt.figure(figsize=(12,6))
# #grid = gridspec.GridSpec(ncols=2,nrows=1,figure=fig)

# plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
# gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
# blur = cv2.GaussianBlur(gray,(7,7),0)
#  # Applied inversed thresh_binary 
# binary = cv2.threshold(blur, 180, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
# kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)

# cont, _  = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# # creat a copy version "test_roi" of plat_image to draw bounding box
# test_roi = plate_image.copy()

# # Initialize a list which will be used to append charater image
# crop_characters = []

# # define standard width and height of character
# digit_w, digit_h = 30, 60

# for c in sort_contours(cont):
#     (x, y, w, h) = cv2.boundingRect(c)
#     ratio = h/w
#     if 1<=ratio<=3.5: # Only select contour with defined ratio
#         if h/plate_image.shape[0]>=0.5: # Select contour which has the height larger than 50% of the plate
#             # Draw bounding box arroung digit number
#             cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)

#             # Sperate number and gibe prediction
#             curr_num = thre_mor[y:y+h,x:x+w]
#             curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
#             _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#             crop_characters.append(curr_num)

# #fig = plt.figure(figsize=(14,4))
# #grid = gridspec.GridSpec(ncols=len(crop_characters),nrows=1,figure=fig)

# # Load model architecture, weight and labels
# json_file = open('models/character_recoginition/MobileNets_character_recognition.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# model = model_from_json(loaded_model_json)
# model.load_weights("models/character_recoginition/License_character_recognition_weight.h5")
# #print("[INFO] Model loaded successfully...")

# labels = LabelEncoder()
# labels.classes_ = np.load('models/character_recoginition/license_character_classes.npy')
# #print("[INFO] Labels loaded successfully...")


# #fig = plt.figure(figsize=(15,3))
# cols = len(crop_characters)
# #grid = gridspec.GridSpec(ncols=cols,nrows=1,figure=fig)

# final_string =  " "
# for i,character in enumerate(crop_characters):
#     title = np.array2string(predict_from_model(character,model,labels))
#     final_string+=title.strip("'[]")

# print(final_string)

In [11]:



def  lpr_process(input_image_path):
    vehicle, LpImg, cor = get_plate(input_image_path)
    plate_image = cv2.convertScaleAbs(LpImg[0], alpha=(255.0))
    gray = cv2.cvtColor(plate_image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(7,7),0)
     # Applied inversed thresh_binary 
    binary = cv2.threshold(blur, 180, 255,cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    kernel3 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    thre_mor = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel3)
    
    cont, _  = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    # creat a copy version "test_roi" of plat_image to draw bounding box
    test_roi = plate_image.copy()
    # Initialize a list which will be used to append charater image
    crop_characters = []
    
    # define standard width and height of character
    digit_w, digit_h = 30, 60
    
    for c in sort_contours(cont):
        (x, y, w, h) = cv2.boundingRect(c)
        ratio = h/w
        if 1<=ratio<=3.5: # Only select contour with defined ratio
            if h/plate_image.shape[0]>=0.5: # Select contour which has the height larger than 50% of the plate
                # Draw bounding box arroung digit number
                cv2.rectangle(test_roi, (x, y), (x + w, y + h), (0, 255,0), 2)
                # Sperate number and gibe prediction
                curr_num = thre_mor[y:y+h,x:x+w]
                curr_num = cv2.resize(curr_num, dsize=(digit_w, digit_h))
                _, curr_num = cv2.threshold(curr_num, 220, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                crop_characters.append(curr_num)
    
    cols = len(crop_characters)
    
    license_plate_string =  ""
    for i,character in enumerate(crop_characters):
        title = np.array2string(predict_characters_from_model(character))
        license_plate_string+=title.strip("'[]")
    
    if len(license_plate_string) >= 3 :
        result = {
            "license_plate_number_detection_status": "Successful",
            "detected_license_plate_number": license_plate_string,
            "input_image_name": input_image_path
        }
        print(json.dumps(result))
    else:
        result = {
            "license_plate_number_detection_status": "Failed",
            "reason": "Not able to read license plate, it could be blur or complex image",
            "input_image_name": input_image_path
        }
        print(json.dumps(result))

def main():
    wpod_net_path = "models/wpod-net.json"
    save_model_h5_to_tf_format(wpod_net_path)
    # Just provide the directory name where the TF *.pb (model file saved_model.pb) file is located
    

   # input_image_path = "dataset/plate5.jpeg"
    
    # for image_id in range(0,11):
    #     input_image_path = "dataset/Cars" + str(image_id) + ".png"
    #     license_plate_string = lpr_process(input_image_path)

    dataset_directory = r'dataset'
    for entry in os.scandir(dataset_directory):
        if (entry.path.endswith(".jpg")
               or entry.path.endswith(".png") or entry.path.endswith(".jpeg")) and entry.is_file():
           license_plate_string = lpr_process(entry.path)
   
if __name__ == "__main__":
    tf_model = keras.models.load_model('models')
    main()




m right(x: 0.564272, y: 0.617365)
Class: 0, top left(x: 0.361433, y: 0.515489), bottom right(x: 0.629687, y: 0.614930)
Class: 0, top left(x: 0.356347, y: 0.506057), bottom right(x: 0.602323, y: 0.614706)
Class: 0, top left(x: 0.374368, y: 0.504776), bottom right(x: 0.647478, y: 0.621504)
Class: 0, top left(x: 0.358200, y: 0.508036), bottom right(x: 0.640424, y: 0.621481)
Class: 0, top left(x: 0.381185, y: 0.500543), bottom right(x: 0.647044, y: 0.614535)
Class: 0, top left(x: 0.447282, y: 0.509761), bottom right(x: 0.646515, y: 0.621677)
Class: 0, top left(x: 0.359419, y: 0.513210), bottom right(x: 0.617318, y: 0.613972)
Class: 0, top left(x: 0.355587, y: 0.506743), bottom right(x: 0.619038, y: 0.615845)
Class: 0, top left(x: 0.393910, y: 0.508934), bottom right(x: 0.667315, y: 0.622645)
Class: 0, top left(x: 0.440592, y: 0.509504), bottom right(x: 0.685939, y: 0.622075)
Class: 0, top left(x: 0.374167, y: 0.508979), bottom right(x: 0.654405, y: 0.622600)
Class: 0, top left(x: 0.358326,

In [12]:
# for image_name in range(0,11):
#     path = "dataset/Cars" + str(image_name) + ".png"
#     print(path)


import os

directory = r'dataset'
for entry in os.scandir(directory):
    if (entry.path.endswith(".jpg")
            or entry.path.endswith(".png") or entry.path.endswith(".jpeg")) and entry.is_file():
        print(entry.path)

dataset/plate4.jpeg
dataset/plate5.jpeg
dataset/Cars3.png
dataset/Cars2.png
dataset/plate2.jpeg
dataset/Cars0.png
dataset/Cars1.png
dataset/plate6.png
dataset/Cars5.png
dataset/Cars4.png
dataset/plate3.jpeg
dataset/Cars6.png
dataset/Cars9.png
dataset/Cars8.png
dataset/plate1.jpeg
